In [17]:
import math

In [30]:
class Board:
    def __init__(self):
        self.board = [['','',''],
                      ['','',''],
                      ['','','']]
        self.values = {'X': 10, 'D': 0, 'O': -10}
        self.to_move = 'X'
        self.state = ''
        self.occupied = 0

    def get_board(self):
        return self.board

    def available(self,x,y):
        try:
            return self.board[x][y] == ''
        except IndexError:
            print("Out of range")
    
    def __move(self,p,x,y):
        if (self.state != ''):
            return "Game is over!"

        try:
            if self.available(x,y) and p == self.to_move:
                self.board[x][y] = p
                self.occupied += 1
                self.state = self.__game_over(p,x,y)
                if self.state != '':
                    print(f'Game is over. {self.state} won.' if self.state != 'D' else "Game is over. Draw")
                    return
                self.to_move = 'X' if p == 'O' else 'O'

                return True
            return False
        except IndexError:
            print("Out of range")

    def x_move(self,x,y):
        return self.__move('X', x, y)
    
    def o_move(self,x,y):
        return self.__move('O', x, y)

    def print_board(self):
        j = 0
        print('  ' + '0' + ' '*2 + '1' + ' '*2 + '2')
        for i in self.board:
            print(str(j)+' |' + '|'.join( [j if j != '' else ' ' for j in i] ) + '|' )
            j+=1

    def __column(self,p,x,y):
        for i in range(3):
            if self.board[i][y] != p:
                return False
        return True

    def __row(self,p,x,y):
        for i in range(3):
            if self.board[x][i] != p:
                return False
        return True

    def __diags(self,p,x,y):
        # The coordinates of the diagonals sum up to an even number
        if (x+y) % 2 != 0:
            return False
        return self.__main_diag(p,x,y) or self.__anti_diag(p,x,y)

    def __main_diag(self,p,x,y):
        # 00 11 22
        for i in range(3):
            if self.board[i][i] != p:
                return False
        return True

    def __anti_diag(self,p,x,y):
        # 02 11 20
        for i, j in zip(range(3), range(2, -1, -1)):
            if self.board[i][j] != p:
                return False
        return True

    # Taking into account the last move
    def __game_over(self,p,x,y):
        if self.__column(p,x,y) or self.__row(p,x,y) or self.__diags(p,x,y):
            return p
        elif self.occupied == 9:
            # one might win on the last move
            return 'D'
        return ''


    def ai_move(self):
        # AI is the minimizer
        move = (-1,-1)
        score = math.inf

        # For each possible move on the board
        for i in range(3):
            for j in range(3):
                if self.available(i,j):
                    # Make the move
                    self.board[i][j] = 'O'
                    self.occupied += 1

                    # Minimax
                    move_value = self.minimax(True, 0, i, j, -math.inf, math.inf)

                    # Make a decision
                    if move_value < score:
                        score = move_value
                        move = (i,j)

                    # Undo the move
                    self.board[i][j] = ''
                    self.occupied -= 1

        return self.o_move(move[0], move[1])

    def minimax(self, maximizing, depth, i, j, alpha, beta):
        # The last move was made by the player who called minimax
        # If we're maximizing, last move was O
        last_move = ''
        if maximizing:
            last_move = 'O'
        else:
            last_move = 'X'

        # Find the result of the game
        result = self.__game_over(last_move, i, j)

        # If the game is decided
        if result != '':
            return self.values[result]

        # If maximizing
        if maximizing:
            score = -math.inf
            
            for r in range(3):
                for c in range(3):
                    if self.available(r,c):
                        # Make the move
                        self.board[r][c] = 'X'
                        self.occupied += 1

                        # Minimax
                        score = max(score, self.minimax(not maximizing, depth+1, r, c, alpha, beta))

                        # Undo the move
                        self.board[r][c] = ''
                        self.occupied -= 1

                        # Alpha-Beta Pruning
                        alpha = max(alpha, score)

                        if beta <= alpha:
                            break

            return score-depth
        # If minimizing
        else:
            score = math.inf

            for r in range(3):
                for c in range(3):
                    if self.available(r,c):
                        # Make the move
                        self.board[r][c] = 'O'
                        self.occupied += 1

                        # Minimax
                        score = min(score, self.minimax(not maximizing, depth+1, r, c, alpha, beta))

                        # Undo the move
                        self.board[r][c] = ''
                        self.occupied -= 1

                        # Alpha-Beta Pruning
                        beta = min(beta, score)

                        if beta <= alpha:
                            break

            return score+depth

In [31]:
B = Board()
B.print_board()

  0  1  2
0 | | | |
1 | | | |
2 | | | |


In [32]:
B = Board()
while (B.state == ''):
    move = input("Move: ")
    move = move.split(' ')
    B.x_move(int(move[0]), int(move[1]))
    B.ai_move()
    B.print_board()

Move: 0 0
  0  1  2
0 |X| | |
1 | |O| |
2 | | | |
Move: 2 2
  0  1  2
0 |X|O| |
1 | |O| |
2 | | |X|
Move: 2 1
  0  1  2
0 |X|O| |
1 | |O| |
2 |O|X|X|
Move: 0 2
  0  1  2
0 |X|O|X|
1 | |O|O|
2 |O|X|X|
Move: 1 0
Game is over. Draw
  0  1  2
0 |X|O|X|
1 |X|O|O|
2 |O|X|X|
